In [1]:
import os
import time
import datetime
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader, TensorDataset, SequentialSampler, RandomSampler
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

from utils_data import Vectorizer, HeadQA, HeadQA_IR, clean_words, parse_dataset, parse_ir_dataset, random_oversamplig, save_dataset_to_pickle, load_dataset_from_pickle
from utils_data import random_undersampling
from training import evaluate, evaluate_better, train_ir, validate_ir, evaluator_ir, load_embeddings_from_file, make_embedding_matrix
from training import get_optimizer

import transformers
from transformers.optimization import AdamW
from transformers import BertForSequenceClassification, BertConfig, BertTokenizer, BertModel, BertForMaskedLM

from ir_models import LSTM_QA

%matplotlib inline
%load_ext autoreload
%autoreload 2

C:\Users\tec005m\Anaconda3\envs\afi\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
from datasets import load_dataset

data_es = load_dataset('head_qa', 'es' )

Reusing dataset head_qa (C:\Users\tec005m\.cache\huggingface\datasets\head_qa\es\1.1.0\473dc5357942a3ff52963bd73cad0d167bd1bbc1ca5ca0732ee7372b480dd735)


In [3]:
training, validation, testing = data_es['train'], data_es['validation'], data_es['test']

In [4]:
training_instances = load_dataset_from_pickle('../data/training_ir.pickle')
validation_instances = load_dataset_from_pickle('../data/validation_ir.pickle')
testing_instances = load_dataset_from_pickle('../data/testing_ir.pickle')
mixed_training = load_dataset_from_pickle('../data/mixed_oversampling_training_ir.pickle')

In [5]:
vectorizer = Vectorizer.vectorize_ir_dataset(mixed_training)

In [6]:
vocab = vectorizer.sentence_vocab
label_vocab = vectorizer.label_vocab

In [7]:
trainset = HeadQA_IR(instances=mixed_training, vectorizer=vectorizer, right_padding=False, max_length=15)
validset = HeadQA_IR(instances=validation_instances, vectorizer=vectorizer, right_padding=False, max_length=15)
testset = HeadQA_IR(instances=testing_instances, vectorizer=vectorizer, right_padding=False, max_length=15)

In [8]:
batch_size = 32
train_dt = DataLoader(trainset, batch_size=batch_size,drop_last=True)
valid_dt = DataLoader(validset, batch_size=batch_size,drop_last=True)
test_dt = DataLoader(testset, batch_size=batch_size,drop_last=True)

In [9]:
word_to_idx = load_dataset_from_pickle('trained_models/biomedical_embeddings/word_to_index_ir.pickle')
embeddings = load_dataset_from_pickle('trained_models/biomedical_embeddings/wordvectors_ir.pickle')

In [10]:
embedding_file = "trained_models/biomedical_embeddings/Scielo_wiki_FastText300.vec"
words = vocab.vocab2index.keys()
embedding_matrix = make_embedding_matrix(embedding_file, list(words), word_to_idx, embeddings)

In [11]:
torch.random.manual_seed(42)
model = LSTM_QA(vocab_size=len(vocab), hidden_size=64, x_size=trainset.max_length, n_classes=1, embedding_size=300,
               pretrained_embeddings=embedding_matrix)
optimizer = get_optimizer(model, lr = 0.001, wd = 1e-5)

Loading pretrained embeddings...


C:\Users\tec005m\Anaconda3\envs\afi\lib\site-packages\torch\nn\modules\rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [12]:
training_results = train_ir(model, optimizer, train_dt, valid_dt, validate_ir, epochs=50)

C:\Users\tec005m\Anaconda3\envs\afi\lib\site-packages\torch\nn\functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch 0 train loss  0.3661 valid loss 0.059 and accuracy 0.2500
Epoch 1 train loss  0.4406 valid loss 0.040 and accuracy 0.2500
Epoch 2 train loss  0.4347 valid loss 0.034 and accuracy 0.2500
Epoch 3 train loss  0.4212 valid loss 0.035 and accuracy 0.2500
Epoch 4 train loss  0.4273 valid loss 0.034 and accuracy 0.2500
Epoch 5 train loss  0.4233 valid loss 0.035 and accuracy 0.2500
Epoch 6 train loss  0.4236 valid loss 0.035 and accuracy 0.2500
Epoch 7 train loss  0.4241 valid loss 0.034 and accuracy 0.2500
Epoch 8 train loss  0.4325 valid loss 0.033 and accuracy 0.2500
Epoch 9 train loss  0.4217 valid loss 0.034 and accuracy 0.2500
Epoch 10 train loss  0.4263 valid loss 0.034 and accuracy 0.2500
Epoch 11 train loss  0.4234 valid loss 0.034 and accuracy 0.2500
Epoch 12 train loss  0.4211 valid loss 0.034 and accuracy 0.2500
Epoch 13 train loss  0.4205 valid loss 0.035 and accuracy 0.2500
Epoch 14 train loss  0.4281 valid loss 0.034 and accuracy 0.2500
Epoch 15 train loss  0.4299 valid l

In [13]:
acc, points = evaluate(model, validation, trainset.encode, evaluator_ir)
acc, points

(tensor([0.2599]), 54)

In [14]:
acc, points = evaluate(model, testing, trainset.encode, evaluator_ir)
acc, points

(tensor([0.2746]), 270)

In [15]:
save_dataset_to_pickle('results_v2/train_results_lstm_qa.pickle', training_results)
training_results = load_dataset_from_pickle('results_v2/train_results_lstm_qa.pickle')

In [16]:
model_path = os.getcwd() + '/trained_models_v2/lstm_qa'
torch.save(model.state_dict(), model_path)

In [17]:
model = LSTM_QA(vocab_size=len(vocab), hidden_size=64, x_size=trainset.max_length, n_classes=1, embedding_size=300,
               pretrained_embeddings=embedding_matrix)
model.load_state_dict(torch.load(model_path))
model.eval()

Loading pretrained embeddings...


LSTM_QA(
  (emb): Embedding(28821, 300, padding_idx=0)
  (dropout): Dropout(p=0.5, inplace=False)
  (lstm): LSTM(300, 64, batch_first=True, dropout=0.5, bidirectional=True)
  (cosine): CosineSimilarity()
  (linear): Linear(in_features=128, out_features=64, bias=True)
  (linear1): Linear(in_features=64, out_features=1, bias=True)
)

In [18]:
acc, points, acc_list, points_list = evaluate_better(model, validation, trainset.encode, evaluator_ir)
acc, points, acc_list, points_list

(0.25986475,
 9.0,
 [tensor(0.2699),
  tensor(0.2304),
  tensor(0.2667),
  tensor(0.2814),
  tensor(0.2301),
  tensor(0.2807)],
 [18, -18, 15, 29, -18, 28])

In [19]:
acc, points, acc_list, points_list = evaluate_better(model, testing, trainset.encode, evaluator_ir)
acc, points, acc_list, points_list

(0.27463463,
 22.5,
 [tensor(0.2675),
  tensor(0.3049),
  tensor(0.2895),
  tensor(0.2457),
  tensor(0.2565),
  tensor(0.3593),
  tensor(0.2522),
  tensor(0.2759),
  tensor(0.2445),
  tensor(0.2511),
  tensor(0.2489),
  tensor(0.2996)],
 [16, 49, 36, -4, 6, 101, 2, 24, -5, 1, -1, 45])